# Импорты

In [5]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')

# Пункт - 1 - baseline (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

```
# ниже пример с пояснением по валидации

train = pd.read_csv(...)
test = pd.read_csv(...)
```
```
train.describe()
...
```
```
test.describe()
...
```

Посмотрев на train и test выборки я выбрал схему валидации TimeSeriesSplit с кол-вом разбиений = 5 и gap = 0 между train и val выборками. Этот выбор основан на следующем
1. ...
2. ...
3. ...

```
# далее разбиение на train и val, которое мы далее будем использовать во всех пунктах задания
# если выбранная схема валидации может возвращать случайные разбиения, то необходимо во всех пунктах задания зафиксировать один и тот же random_state
cv = TimeSeriesSplit(...)

# обратите внимание, в test выборке House Prices отсутствует колонка SalePrice, поэтому эту выборку мы НЕ используем далее
for train_idx, test_idx in cv.split(train):
    ...
```

In [3]:
# Загрузка данных
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Пункт 1: Анализ отличий train и test выборки и выбор схемы кросс-валидации
print("=== Анализ train и test выборок ===")
print(f"Размер train: {train.shape}")
print(f"Размер test: {test.shape}")
print(f"Колонки train: {train.columns.tolist()}")
print(f"Колонки test: {test.columns.tolist()}")

# Проверяем наличие временных меток или последовательной структуры
print("\nПервые 5 строк train:")
print(train.head())
print("\nПервые 5 строк test:")
print(test.head())

# Анализ различий в распределениях численных признаков (если есть общие признаки)
numeric_columns_train = train.select_dtypes(include=[np.number]).columns
numeric_columns_test = test.select_dtypes(include=[np.number]).columns
common_numeric = set(numeric_columns_train) & set(numeric_columns_test)

print(f"\nОбщие численные признаки: {common_numeric}")

# Выбор схемы валидации - TimeSeriesSplit как указано в задании
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)
print(f"\nВыбрана схема валидации: TimeSeriesSplit с n_splits=5")

# Пункт 2: Подготовка данных и обучение модели kNN
# Выбираем только численные признаки
X_train = train.select_dtypes(include=[np.number])
X_test = test.select_dtypes(include=[np.number])

# Заполняем пропущенные значения нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

print(f"\nЧисленные признаки после обработки:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

# Проверяем, есть ли целевая переменная
# Предположим, что целевая переменная называется 'target' или аналогично
target_column = None
for col in ['target', 'y', 'price', 'value']:
    if col in train.columns:
        target_column = col
        break

if target_column is None:
    print("Целевая переменная не найдена. Используем первую численную колонку как пример.")
    target_column = X_train.columns[0]

y_train = train[target_column]

print(f"Целевая переменная: {target_column}")

# Пункт 3: Оценка качества модели с кросс-валидацией
mse_scores_train = []
mse_scores_val = []
mape_scores_train = []
mape_scores_val = []

print("\n=== Кросс-валидация ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Обучение модели kNN
    model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
    model.fit(X_train_fold, y_train_fold)
    
    # Предсказания
    y_pred_train = model.predict(X_train_fold)
    y_pred_val = model.predict(X_val_fold)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train.append(mse_train)
    mse_scores_val.append(mse_val)
    mape_scores_train.append(mape_train)
    mape_scores_val.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train), np.mean(mse_scores_val)],
    'MSE_median': [np.median(mse_scores_train), np.median(mse_scores_val)],
    'MAPE_mean': [np.mean(mape_scores_train), np.mean(mape_scores_val)],
    'MAPE_median': [np.median(mape_scores_train), np.median(mape_scores_val)]
})

print("\n=== Итоговые результаты ===")
print(results)

# Дополнительно: обучение на всем train и предсказание для test
print("\n=== Обучение на полном train датасете ===")
final_model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
final_model.fit(X_train, y_train)

# Предсказания для train (для оценки на всех данных)
y_pred_full_train = final_model.predict(X_train)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"MSE на полном train: {mse_full_train:.4f}")
print(f"MAPE на полном train: {mape_full_train:.4f}")

# Если в test есть те же признаки, можем сделать предсказания
if X_test.shape[1] == X_train.shape[1]:
    test_predictions = final_model.predict(X_test)
    print(f"\nСделано предсказаний для test: {len(test_predictions)}")
else:
    print(f"\nПредсказания для test невозможны: различное количество признаков")

=== Анализ train и test выборок ===
Размер train: (1460, 81)
Размер test: (1459, 80)
Колонки train: ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDr

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Анализ train и test выборок

1. Структура данных:

Train выборка содержит 1460 наблюдений и 81 признак

Test выборка содержит 1459 наблюдений и 80 признаков

2. Единственное различие - в test выборке отсутствует целевая переменная 'SalePrice'

Выбор схемы валидации:

Выбрана схема TimeSeriesSplit с n_splits=5

Обоснование выбора:

- Данные содержат временные метки ('YrSold', 'MoSold'), что указывает на временную природу данных

- TimeSeriesSplit сохраняет временной порядок данных, что важно для временных рядов

- Схема соответствует требованию задания использовать 5 разбиений

### Интерпретация результатов

1. Качество модели:

MSE на валидационной выборке выше, чем на тренировочной, что ожидаемо и свидетельствует об отсутствии переобучения

MAPE ≈ 8.7-10.4% показывает, что модель в среднем ошибается на 8.7-10.4% от реальной цены

2. Эффективность baseline:

Модель kNN с параметрами по умолчанию показывает разумное baseline качество

Разница между train и validation ошибками умеренная, что говорит о хорошей обобщающей способности

# Пункт - 2 - baseline + scaling (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Загрузка данных
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Выбор численных признаков и целевой переменной
X_train = train.select_dtypes(include=[np.number]).drop('SalePrice', axis=1)
X_test = test.select_dtypes(include=[np.number])
y_train = train['SalePrice']

# Заполнение пропущенных значений нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

mse_scores_train = []
mse_scores_val = []
mape_scores_train = []
mape_scores_val = []

print("=== Кросс-валидация с масштабированием ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    # Разделение данных
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Масштабирование признаков
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled = scaler.transform(X_val_fold)
    
    # Обучение модели kNN
    model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
    model.fit(X_train_scaled, y_train_fold)
    
    # Предсказания
    y_pred_train = model.predict(X_train_scaled)
    y_pred_val = model.predict(X_val_scaled)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train.append(mse_train)
    mse_scores_val.append(mse_val)
    mape_scores_train.append(mape_train)
    mape_scores_val.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

# Итоговые результаты
results_scaled = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train), np.mean(mse_scores_val)],
    'MSE_median': [np.median(mse_scores_train), np.median(mse_scores_val)],
    'MAPE_mean': [np.mean(mape_scores_train), np.mean(mape_scores_val)],
    'MAPE_median': [np.median(mape_scores_train), np.median(mape_scores_val)]
})

print("\n=== Итоговые результаты с масштабированием ===")
print(results_scaled)

# Обучение на полном датасете с масштабированием
scaler_full = StandardScaler()
X_train_scaled_full = scaler_full.fit_transform(X_train)

final_model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
final_model.fit(X_train_scaled_full, y_train)

y_pred_full_train = final_model.predict(X_train_scaled_full)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"\nMSE на полном train с масштабированием: {mse_full_train:.4f}")
print(f"MAPE на полном train с масштабированием: {mape_full_train:.4f}")


mse_scores_train_scaled.append(mse_train)
mse_scores_val_scaled.append(mse_val)
mape_scores_train_scaled.append(mape_train)
mape_scores_val_scaled.append(mape_val)

results_scaled = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_scaled), np.mean(mse_scores_val_scaled)],
    'MSE_median': [np.median(mse_scores_train_scaled), np.median(mse_scores_val_scaled)],
    'MAPE_mean': [np.mean(mape_scores_train_scaled), np.mean(mape_scores_val_scaled)],
    'MAPE_median': [np.median(mape_scores_train_scaled), np.median(mape_scores_val_scaled)]
})

print("Результаты с масштабированием:")
print(results_scaled)

# Сравнение
print("\n=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===")
print(f"Улучшение MSE (validation): {((np.mean(mse_scores_val_no_scale) - np.mean(mse_scores_val_scaled)) / np.mean(mse_scores_val_no_scale) * 100):.1f}%")
print(f"Улучшение MAPE (validation): {((np.mean(mape_scores_val_no_scale) - np.mean(mape_scores_val_scaled)) / np.mean(mape_scores_val_no_scale) * 100):.1f}%")

=== Кросс-валидация с масштабированием ===
Fold 1: MSE_train=985692905.9716, MSE_val=1373464634.2222, MAPE_train=0.1099, MAPE_val=0.1383
Fold 2: MSE_train=802301847.2868, MSE_val=1811468045.9666, MAPE_train=0.1023, MAPE_val=0.1503
Fold 3: MSE_train=1028442514.2724, MSE_val=1315291581.4049, MAPE_train=0.1080, MAPE_val=0.1220
Fold 4: MSE_train=982935533.2474, MSE_val=1793271622.2188, MAPE_train=0.1062, MAPE_val=0.1223
Fold 5: MSE_train=1046843199.0792, MSE_val=1760599594.6324, MAPE_train=0.1045, MAPE_val=0.1277

=== Итоговые результаты с масштабированием ===
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  9.692432e+08  9.856929e+08   0.106170     0.106219
1  Validation  1.610819e+09  1.760600e+09   0.132105     0.127713

MSE на полном train с масштабированием: 1054260585.9596
MAPE на полном train с масштабированием: 0.1034


NameError: name 'mse_scores_train_scaled' is not defined

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 3 - baseline + categorical features (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 4 - baseline + scaling + categorical features (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 5 лучший дизайн + power transform (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 6 Все предыдущие приемы п1-п5 + power transform (target)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 7 лучший дизайн + обработка выбросов (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 8 лучший дизайн + подбор гиперпараметров (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ